In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Data from [https://cimis.water.ca.gov/Default.aspx](https://cimis.water.ca.gov/Default.aspx) and [https://et.water.ca.gov/Rest/Index](https://et.water.ca.gov/Rest/Index)

In [20]:
import sys
sys.path.append("../loaders/weather_station")

In [21]:
import weather_reports

In [22]:
source = pd.read_csv("../data/source.csv", parse_dates=True)
source[["start", "end"]] = source[["start", "end"]].astype("datetime64[ns]")

### Import weather station index

In [23]:
ws = pd.read_csv("../data/weather/weather_stations.csv", parse_dates=True)
ws.columns

Index(['StationNbr', 'Name', 'City', 'RegionalOffice', 'County', 'ConnectDate',
       'DisconnectDate', 'IsActive', 'IsEtoStation', 'Elevation',
       'GroundCover', 'HmsLatitude', 'HmsLongitude', 'ZipCodes', 'SitingDesc'],
      dtype='object')

In [24]:
ws[ws.County.str.contains("Yolo")]

,StationNbr,Name,City,RegionalOffice,County,ConnectDate,DisconnectDate,IsActive,IsEtoStation,Elevation,GroundCover,HmsLatitude,HmsLongitude,ZipCodes,SitingDesc
5,6,Davis,Davis,North Central Region Office,Yolo,7/17/1982,12/31/2050,True,True,60,Grass,38º32'8N / 38.535694,-121º46'35W / -121.776360,"['95618', '95617', '95616']",NaN
26,27,Zamora,Woodland,North Central Region Office,Yolo,12/5/1982,1/20/2006,False,True,50,Grass,38º48'32N / 38.808758,-121º54'27W / -121.907540,['95698'],NaN
154,155,Bryte (experimental),West Sacramento,North Central Region Office,Yolo,12/10/1998,12/31/2050,True,False,40,Bare,38º35'57N / 38.599158,-121º32'25W / -121.540410,"['95799', '95798', '95691', '95605']",NaN
195,196,Esparto,Esparto,North Central Region Office,Yolo,4/15/2005,7/15/2017,False,True,174,Grass,38º41'30N / 38.691786,-122º0'50W / -122.013810,"['95607', '95627', '95653', '95695']",NaN
225,226,Woodland,Woodland,North Central Region Office,Yolo,5/12/2011,12/31/2050,True,True,82,Grass,38º40'22N / 38.672722,-121º48'42W / -121.811720,"['95695', '95776', '95627']",NaN


### Match Weather Station API against SCE Weather Station names

In [11]:
sce_ws = source["Weather Station"].unique()
api_ws = ws.Name
matches = {}
for n in api_ws:
    for w in sce_ws:
        if n.lower() in w.lower() and n not in matches:
            matches[n]=w

In [12]:
matches

{'Davis': 'SCE Mount Davis',
 'Soledad': 'SCE Soledad Canyon Rd',
 'Santa Paula': 'SCE Santa Paula',
 'Escondido': 'SCE Escondido Canyon',
 'Ramona': 'SCE Ramona Expy',
 'Moorpark': 'SCE Moorpark',
 'Moreno Valley': 'SCE Moreno Valley Fwy',
 'Williams': 'SCE Williams Canyon Rd',
 'Highland': 'HIGHLAND SPRINGS RAW',
 'Chino': 'SCE Chino Hills 3'}

In [13]:
ws_matches = ws[ws.Name.isin(matches.keys())].copy()
ws_matches.Name = ws_matches.Name.apply(lambda s: matches[s])
source_ws = source.merge(ws_matches, left_on="Weather Station", right_on="Name")

### Prepare query for Weather Station API using matched Weather Stations

In [14]:
ws_query = source_ws[["StationNbr", "start", "end"]]
ws_query = ws_query[~ws_query.duplicated()]

In [15]:
# look at data the day before/after the event
one_day = pd.Timedelta(days=1)
one_month = pd.Timedelta(days=30)
ws_query.start = ws_query.start - one_month
ws_query.end = ws_query.end + one_month

In [16]:
query = ws_query.astype("str").values
query

array([['6', '2020-10-30', '2020-12-29'],
       ['6', '2020-11-16', '2021-01-23'],
       ['6', '2020-11-04', '2021-01-13'],
       ['6', '2020-12-13', '2021-02-20'],
       ['98', '2020-10-30', '2020-12-29'],
       ['238', '2020-10-30', '2020-12-29'],
       ['58', '2020-10-30', '2020-12-29'],
       ['198', '2020-10-30', '2020-12-29'],
       ['58', '2020-12-13', '2021-02-20'],
       ['198', '2020-12-13', '2021-02-20'],
       ['28', '2020-10-30', '2020-12-29'],
       ['217', '2020-10-30', '2020-12-29'],
       ['251', '2020-11-04', '2021-01-13'],
       ['250', '2020-11-04', '2021-01-13'],
       ['74', '2020-11-04', '2021-01-13'],
       ['74', '2020-12-13', '2021-02-20'],
       ['255', '2020-12-13', '2021-02-20']], dtype=object)

In [23]:
ws.loc[ws.StationNbr.isin(query[:,0]), ["StationNbr", "Name", "City"]]

,StationNbr,Name,City
5,6,Davis,Davis
27,28,Soledad,Soledad
57,58,Santa Paula,Santa Paula
73,74,Escondido,Escondido
97,98,Ramona,Ramona
197,198,Santa Paula,Santa Paula
216,217,Moorpark,Moorpark
237,238,Moreno Valley,Moreno Valley
249,250,Williams,Williams
250,251,Highland,Highland


In [58]:
start = "2020-12-01"
end = "2021-02-01"
full_query = []
for s in ws[ws.IsActive==True].StationNbr:
    full_query.append([s, start, end])

[[2, '2020-12-01', '2021-02-01'],
 [5, '2020-12-01', '2021-02-01'],
 [6, '2020-12-01', '2021-02-01'],
 [7, '2020-12-01', '2021-02-01'],
 [12, '2020-12-01', '2021-02-01'],
 [13, '2020-12-01', '2021-02-01'],
 [15, '2020-12-01', '2021-02-01'],
 [35, '2020-12-01', '2021-02-01'],
 [39, '2020-12-01', '2021-02-01'],
 [41, '2020-12-01', '2021-02-01'],
 [43, '2020-12-01', '2021-02-01'],
 [44, '2020-12-01', '2021-02-01'],
 [47, '2020-12-01', '2021-02-01'],
 [52, '2020-12-01', '2021-02-01'],
 [54, '2020-12-01', '2021-02-01'],
 [56, '2020-12-01', '2021-02-01'],
 [57, '2020-12-01', '2021-02-01'],
 [62, '2020-12-01', '2021-02-01'],
 [64, '2020-12-01', '2021-02-01'],
 [68, '2020-12-01', '2021-02-01'],
 [70, '2020-12-01', '2021-02-01'],
 [71, '2020-12-01', '2021-02-01'],
 [75, '2020-12-01', '2021-02-01'],
 [77, '2020-12-01', '2021-02-01'],
 [78, '2020-12-01', '2021-02-01'],
 [80, '2020-12-01', '2021-02-01'],
 [83, '2020-12-01', '2021-02-01'],
 [84, '2020-12-01', '2021-02-01'],
 [87, '2020-12-01', '202

### Inspect results, output to CSV

In [7]:
wr = pd.read_csv("../data/weather/weather_report_1620408982.771627.csv")
wr.columns

Index(['Date', 'Julian', 'Hour', 'Station', 'Standard', 'ZipCodes', 'Scope',
       'HlyAirTmp (F)', 'HlyDewPnt (F)', 'HlyEto (in)', 'HlyNetRad (Ly/day)',
       'HlyAsceEto (in)', 'HlyAsceEtr (in)', 'HlyPrecip (in)', 'HlyRelHum (%)',
       'HlyResWind (MPH)', 'HlySoilTmp (F)', 'HlySolRad (Ly/day)',
       'HlyVapPres (mBars)', 'HlyWindDir (°)', 'HlyWindSpd (MPH)'],
      dtype='object')

In [9]:
wind_max = wr[["Station", "HlyWindSpd (MPH)"]].groupby("Station").max()
wind_max = wind_max[wind_max > 31].dropna()

In [10]:
ws[ws.StationNbr.isin(wind_max.index)]

,StationNbr,Name,City,RegionalOffice,County,ConnectDate,DisconnectDate,IsActive,IsEtoStation,Elevation,GroundCover,HmsLatitude,HmsLongitude,ZipCodes,SitingDesc
120,121,Dixon,Dixon,North Central Region Office,Solano,9/20/1994,12/31/2050,True,True,37,Grass,38º24'56N / 38.415564,-121º47'13W / -121.786910,['95620'],NaN
139,140,Twitchell Island,Rio Vista,North Central Region Office,Sacramento,10/8/1997,12/31/2050,True,True,0,Grass,38º7'18N / 38.121739,-121º40'28W / -121.674455,"['94571', '95641']",NaN
211,212,Hastings Tract East,Dixon,North Central Region Office,Solano,10/20/2009,12/31/2050,True,True,7,Grass,38º16'41N / 38.278056,-121º44'28W / -121.741110,"['94571', '95620']",NaN
247,248,Holt,Stockton,North Central Region Office,San Joaquin,7/1/2016,12/31/2050,True,True,-3,Grass,37º55'55N / 37.932072,-121º23'48W / -121.396661,"['95206', '95234', '95203', '95219']",NaN
260,261,Gazelle,Gazelle,Northern Region Office,Siskiyou,9/18/2019,12/31/2050,True,True,2743,Grass,41º32'2N / 41.533989,-122º31'56W / -122.532279,"['96034', '96038', '96064', '96094']",NaN


In [25]:
wr2021 = pd.read_csv("../data/weather/weather_report_DectoJan2021.csv", parse_dates=True)
wr2020 = pd.read_csv("../data/weather/weather_report_DectoJan2020.csv", parse_dates=True)
wr2019 = pd.read_csv("../data/weather/weather_report_DectoJan2019.csv", parse_dates=True)

/Users/tomityrrell/opt/anaconda3/envs/gluon-ts/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
wrDecJan = pd.concat([wr2019, wr2020, wr2021])
wrDecJan.Date = pd.to_datetime(wrDecJan.Date)

In [30]:
wrDecJan.to_csv("../data/weather/weather_report_DecJan.csv", index=False)

In [31]:
wrDecJan.head()

,Date,Julian,Hour,Station,Standard,ZipCodes,Scope,HlyAirTmp (F),HlyDewPnt (F),HlyEto (in),...,HlyAsceEto (in),HlyAsceEtr (in),HlyPrecip (in),HlyRelHum (%),HlyResWind (MPH),HlySoilTmp (F),HlySolRad (Ly/day),HlyVapPres (mBars),HlyWindDir (°),HlyWindSpd (MPH)
0,2018-12-01,335,100,2,english,93624,hourly,41.6,39.8,0.0,...,0.0,NaN,0.0,93.0,2.6,57.4,0.0,8.3,201.0,2.7
1,2018-12-01,335,200,2,english,93624,hourly,43.9,39.5,0.0,...,0.0,NaN,0.0,85.0,2.6,57.3,0.0,8.2,239.0,3.4
2,2018-12-01,335,300,2,english,93624,hourly,41.7,39.1,0.0,...,0.0,NaN,0.0,90.0,1.2,57.1,0.0,8.1,5.0,1.8
3,2018-12-01,335,400,2,english,93624,hourly,39.2,37.9,0.0,...,0.0,NaN,0.0,95.0,1.0,57.0,0.0,7.7,127.0,1.7
4,2018-12-01,335,500,2,english,93624,hourly,37.0,36.2,0.0,...,0.0,NaN,0.0,97.0,2.0,56.8,0.0,7.2,180.0,2.0


In [40]:
crossings = wrDecJan[wrDecJan["HlyWindSpd (MPH)"] > wrDecJan["HlyRelHum (%)"]]
crossings.Station.value_counts()

220    833
39     600
249    576
258    351
41     268
      ... 
205      1
84       1
209      1
253      1
191      1
Name: Station, Length: 86, dtype: int64

In [44]:
for (d,df) in crossings.groupby("Date"):
    print(d)

2018-12-04 00:00:00
2018-12-09 00:00:00
2018-12-10 00:00:00
2018-12-11 00:00:00
2018-12-12 00:00:00
2018-12-13 00:00:00
2018-12-14 00:00:00
2018-12-15 00:00:00
2018-12-16 00:00:00
2018-12-17 00:00:00
2018-12-18 00:00:00
2018-12-19 00:00:00
2018-12-20 00:00:00
2018-12-21 00:00:00
2018-12-22 00:00:00
2018-12-23 00:00:00
2018-12-24 00:00:00
2018-12-25 00:00:00
2018-12-26 00:00:00
2018-12-27 00:00:00
2018-12-28 00:00:00
2018-12-29 00:00:00
2018-12-30 00:00:00
2018-12-31 00:00:00
2019-01-01 00:00:00
2019-01-02 00:00:00
2019-01-03 00:00:00
2019-01-04 00:00:00
2019-01-05 00:00:00
2019-01-06 00:00:00
2019-01-07 00:00:00
2019-01-08 00:00:00
2019-01-09 00:00:00
2019-01-10 00:00:00
2019-01-11 00:00:00
2019-01-12 00:00:00
2019-01-20 00:00:00
2019-01-21 00:00:00
2019-01-22 00:00:00
2019-01-23 00:00:00
2019-01-24 00:00:00
2019-01-25 00:00:00
2019-01-26 00:00:00
2019-01-27 00:00:00
2019-01-28 00:00:00
2019-01-29 00:00:00
2019-01-30 00:00:00
2019-01-31 00:00:00
2019-02-01 00:00:00
2019-12-05 00:00:00
